## C S 329E HW 7

# Naive Bayes 

## Group 26: Fayed Nassor and Mohamad Minoneshan

For this week's homework we are going explore one new classification technique:

  - Naive Bayes

We are reusing the version of the Melbourne housing data set from HW6, to predict the housing type as one of three possible categories:

  - 'h' house
  - 'u' duplex
  - 't' townhouse

In addition to building our own Naive Bayes classifier, we are going to compare the performace of our classifier to the [Gaussian Naive Bayes Classifier](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) available in the scikit-learn library. 

In [ ]:
# These are the libraries you will use for this assignment
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import calendar
from sklearn.naive_bayes import GaussianNB # The only thing in scikit-learn you can use this assignment

# Starting off loading a training set and setting a variable for the target column, "Type"
df_melb = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/81b236aecee57f6cf65e60afd865d2bb/raw/56ddb53aa90c26ab1bdbfd0b8d8229c8d08ce45a/melb_data_train.csv')
target_col = 'Type'

## Q1 - Fix a column of data to be numeric
If we inspect our dataframe, `df_melb` using the `dtypes` method, we see that the column "Date" is an object.  However, we think this column might contain useful information so we want to convert it to [seconds since epoch](https://en.wikipedia.org/wiki/Unix_time). Use only the exiting imported libraries to create a new column "unixtime". Be careful, the date strings in the file might have some non-uniform formating that you have to fix first.  Print out the min and max epoch time to check your work.  Drop the original "Date" column. 

THESE ARE THE EXACT SAME INSTRUCTIONS FROM HW6! Please take this opportunity to reuse your code (if you got it right last time). 

In [ ]:
# normalize date accepts the date string as shown in the df_melb 'Date' column,
# and returns a data in a standarized format
def standardize_date(d):
    day, month, year = map(int, d.split('/'))
    if len(str(year)) == 2:
        if year > 68:
            year = int('19' + str(year))
        else:
            year = int('20' + str(year))
    time_struct = time.struct_time((year, month, day, 0, 0, 0, -1, -1, -1))
    epoch_time = int(time.mktime(time_struct))
    return epoch_time

In [ ]:
df_melb['Date'] = df_melb['Date'].apply( standardize_date )
df_melb['unixtime'] = df_melb['Date']
df_melb = df_melb.drop(columns="Date")

print("The min unixtime is {:d} and the max unixtime is {:d}".format(df_melb['unixtime'].min(),df_melb['unixtime'].max()))

The min unixtime is 1454544000 and the max unixtime is 1506124800


In [ ]:
df_melb

,Rooms,Type,Price,Distance,Postcode,Bathroom,Car,Landsize,BuildingArea,YearBuilt,unixtime
0,2,h,399000,8.7,3032,1,1.0,904,53.0,1985.0,1462579200
1,3,h,1241000,13.9,3165,1,1.0,643,NaN,NaN,1472342400
2,2,u,550000,3.0,3067,1,1.0,1521,NaN,NaN,1499472000
3,3,u,691000,8.4,3072,1,1.0,170,NaN,NaN,1498262400
4,2,u,657500,4.6,3122,1,1.0,728,73.0,1965.0,1479513600
...,...,...,...,...,...,...,...,...,...,...,...
495,2,t,710000,7.2,3184,1,1.0,0,NaN,1980.0,1501286400
496,2,u,446000,8.0,3040,1,1.0,0,NaN,NaN,1460764800
497,3,h,887000,9.2,3058,1,2.0,560,NaN,NaN,1491609600
498,3,h,1365000,8.0,3040,1,1.0,754,NaN,NaN,1462579200


## Q2 Calculating the prior probabilities
Calculate the prior probabilities for each possible "Type" in `df_melb` and populate a dictionary, `dict_priors`, where the key is the possible "Type" values and the value is the prior probabilities. Show the dictionary. Do not hardcode the possible values of "Type".  Don't forget about [value counts](https://pandas.pydata.org/docs/reference/api/pandas.Series.value_counts.html). 

In [ ]:
counts = df_melb['Type'].value_counts() / len(df_melb)
dict_priors = counts.to_dict()

# show the priors
dict_priors

{'h': 0.452, 'u': 0.418, 't': 0.13}

## Q3 Create a model for the distribution of all of the numeric attributes
For each class, and for each attribute calculate the sample mean and sample standard deviation.  You should store the model in a nested dictionary, `dict_nb_model`, such that `dict_nb_model['h']['Rooms']` is a tuple containing the mean and standard deviation for the target Type 'h' and the attribute 'Rooms'.  Show the model using the `display` function. You should ignore entries that are `NaN` in the mean and [standard deviation](https://pandas.pydata.org/docs/reference/api/pandas.Series.std.html) calculation. 

In [ ]:
dict_nb_model = dict()
for target in dict_priors.keys():
    # your code here
    dict_nb_model[target] = dict()
    for att in df_melb.columns.drop('Type'):
      dict_nb_model[target][att] = (df_melb[df_melb['Type'] == target][att].mean(),df_melb[df_melb['Type'] == target][att].std())

In [ ]:
display(dict_nb_model)

{'h': {'Rooms': (3.269911504424779, 0.725826420112775),
  'Price': (1189022.3451327435, 586296.5794417894),
  'Distance': (12.086725663716816, 7.397501132737295),
  'Postcode': (3103.8982300884954, 98.35750345419703),
  'Bathroom': (1.5619469026548674, 0.6720871086493074),
  'Car': (1.7777777777777777, 0.932759177140425),
  'Landsize': (932.9646017699115, 3830.7934157687173),
  'BuildingArea': (156.2433962264151, 54.62662837301433),
  'YearBuilt': (1954.900826446281, 32.4618763471547),
  'unixtime': (1485717578.761062, 13838562.05060146)},
 'u': {'Rooms': (2.0430622009569377, 0.5908453859944255),
  'Price': (634207.1770334928, 217947.32866736987),
  'Distance': (8.760287081339714, 5.609778714430756),
  'Postcode': (3120.4545454545455, 87.18475679946476),
  'Bathroom': (1.1818181818181819, 0.4222815154866222),
  'Car': (1.1483253588516746, 0.47231993860297056),
  'Landsize': (436.23444976076553, 1394.3403794653257),
  'BuildingArea': (83.85585585585585, 45.95943801516662),
  'YearBuilt'

## Q4 Write a function that calculates the probability of a Gaussian
Given the mean ($\mu$), standard deviation ($\sigma$), and a observed point, `x`, return the probability.  
Use the formula $p(x) = \frac{1}{\sigma \sqrt{2 \pi}} e^{-\frac{1}{2}(\frac{x-\mu}{\sigma})^2}$ ([wiki](https://en.wikipedia.org/wiki/Normal_distribution)).  You should use [numpy's exp](https://numpy.org/doc/stable/reference/generated/numpy.exp.html) function in your solution. 

In [ ]:
def get_p( mu, sigma, x):
    # your code here
    p = np.exp(-0.5*((x-mu)/sigma)**2) / (sigma * np.sqrt(2*np.pi))
    return p

In [ ]:
# Test it. You should get a value of 0.19333405840142462
p = get_p( 0, 2, 0.5)
p

0.19333405840142465

## Q5 Write the Naive Bayes classifier function
The Naive Bayes classifier function, `nb_class`, should take as a parameter the prior probability dictionary, `dict_priors`, the dictionary containing all of the gaussian distribution information for each attribue, `dict_nb_model`, and a single observation row (a series generated from iterrows) of the test dataframe. It should return a single target classification. For this problem, all of our attributes are numeric and modeled as Gaussians, so we don't worry about categorical data. Make sure to skip attributes that do not have a value in the observation.  Do not hardcode the possible classification types. 

In [ ]:
def nb_class( dict_priors, dict_nb_model, observation):
    # your code here  
    max_posterior_prob = -1
    max_posterior_class = None
    
    for target in dict_priors.keys():
        prior_prob = dict_priors[target]
        posterior_prob = prior_prob
        
        for att in dict_nb_model[target].keys():
            if pd.isna(observation[att]):
                continue
            
            mu, sigma = dict_nb_model[target][att]
            x = observation[att]
            p = get_p(mu, sigma, x)
            posterior_prob *= p
        
        if posterior_prob > max_posterior_prob:
            max_posterior_prob = posterior_prob
            max_posterior_class = target
    
    return max_posterior_class

## Q6 Calculate the accuracy using Naive Bayes classifier function on the test set
Load the test set from file, convert date to unix time and drop the date column, classify each row using your `nb_class`, and then show the accuracy on the test set. 

In [ ]:
df_test = pd.read_csv('https://gist.githubusercontent.com/yanyanzheng96/c3d53303cebbd986b166591d19254bac/raw/94eb3b2d500d5f7bbc0441a8419cd855349d5d8e/melb_data_test.csv')
df_test['Date'] = df_test['Date'].apply( standardize_date )
df_test['unixtime'] = df_test['Date']
df_test = df_test.drop(columns="Date")

In [ ]:
predictions = []

for (indx,row) in df_test.iterrows():
    # your code here
    predictions.append(nb_class(dict_priors, dict_nb_model, row))


In [ ]:
acc = (predictions == df_test['Type']).mean()

In [ ]:
# You should get and accuracy of 57.00%
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 57.00%


## Use scikit-learn to do the same thing!

Now we understand the inner workings of the Naive Bayes algorithm, let's compare our results to [scikit-learn's Naive Bayes](https://scikit-learn.org/stable/modules/naive_bayes.html) implementation. Use the [GaussianNB](https://scikit-learn.org/stable/modules/naive_bayes.html#gaussian-naive-bayes) to train using the `df_melb`dataframe and test using the `df_test` dataframe. Remember to split `df_melb` into a `df_X` with the numerical attributes, and a `s_y` with the target column. On the `df_melb` frame you will have to fill the empty attributes via imputation since the scikit-learn library can not handle missing values.  Use the same method you used in the last homework (filling the training data with the mean of the non-nan values). 

Answer the following in a markdown cell: do you think imputation hurt or helped the classifier?

In [ ]:
# Imputation training
dict_imputation = dict()
# your code here
df_X = df_melb.drop("Type", axis=1)
s_y = df_melb["Type"]
for col in df_X.columns:
    if col == 'Type':
        continue
    else:
        dict_imputation[col] = df_X[col].mean()
        df_X[col].fillna(dict_imputation[col], inplace=True)
# Imputation - apply on the test data
# your code here
for col in df_test.columns:
    if col == 'Type':
        continue
    else:
        df_test[col].fillna(dict_imputation[col],inplace=True)

# Seperate the attributes from the target_col
# your code here
X_train = df_X.values
y_train = s_y.values
X_test = df_test.drop("Type", axis=1).values


In [ ]:
gnb = GaussianNB()
# your code here
gnb.fit(X_train, y_train)
y_pred = gnb.predict(X_test)

In [ ]:
acc = (y_pred == df_test['Type']).mean()
print('Accuracy is {:.2f}%'.format(acc*100))

Accuracy is 37.00%


## ANSWER TO "do you think imputation hurt or helped the classifier?" 

#### Answer:
We think it hurt it as apparently our accuracy decreased from 57% to 37%!